In [ ]:
# import os
# import json
# import time
# import pandas as pd
# from unstract.llmwhisperer import LLMWhispererClientV2
# from unstract.llmwhisperer.client_v2 import LLMWhispererClientException

# # Initialize client
# client = LLMWhispererClientV2(api_key=api)

# # Define output directory
# output_dir = "data/output/Quatersdata"
# os.makedirs(output_dir, exist_ok=True)

# # First, verify file paths exist
# pdf_list = [
#     "data/pdfs/Quarter I Ended FY 2025-26.pdf",
#     "data/pdfs/Quarter III Ended FY 2024-25.pdf", 
#     "data/pdfs/Quarter IV & Year Ended FY 2024-25.pdf"
# ]

# print("=== FILE VERIFICATION ===")
# valid_pdfs = []
# for pdf_path in pdf_list:
#     if os.path.exists(pdf_path):
#         file_size = os.path.getsize(pdf_path) / (1024*1024)  # MB
#         print(f"✓ {pdf_path} exists ({file_size:.1f} MB)")
#         valid_pdfs.append(pdf_path)
#     else:
#         print(f"✗ {pdf_path} NOT FOUND")
#         # Try to find it
#         current_dir = os.getcwd()
#         for root, dirs, files in os.walk(current_dir):
#             if pdf_path.split('/')[-1] in files:
#                 found_path = os.path.join(root, pdf_path.split('/')[-1])
#                 print(f"  Found at: {found_path}")
#                 valid_pdfs.append(found_path)
#                 break

# if not valid_pdfs:
#     print("ERROR: No valid PDF files found. Please check paths.")
#     exit(1)

# print(f"\nProcessing {len(valid_pdfs)} valid PDF(s)...")

# # Aggregate results
# all_results = {"documents": [], "processing_summary": {"successful": 0, "failed": 0, "total_pages": 0}}

# for pdf_path in valid_pdfs:
#     doc_name = os.path.basename(pdf_path)
#     doc_id = doc_name.replace('.pdf', '').replace(' ', '_').replace('&', 'and').replace('/', '_')
    
#     print(f"\n--- Processing {doc_name} ---")
    
#     try:
#         # Get file stats
#         file_size_mb = os.path.getsize(pdf_path) / (1024*1024)
#         print(f"File size: {file_size_mb:.1f} MB")
        
#         # Check your usage first (if available)
#         try:
#             usage = client.get_usage_info() if hasattr(client, 'get_usage_info') else None
#             if usage:
#                 print(f"API Usage: {usage}")
#         except:
#             pass
        
#         # Process with more detailed parameters
#         result = client.whisper(
#             file_path=pdf_path,
#             wait_for_completion=True,
#             wait_timeout=600,  # 10 minutes timeout
#             output_mode="layout_preserving",
#             pages_to_extract="",  # All pages
#             mode="high_quality",
#             # Add these for better debugging
#             page_seperator="<<<PAGE>>>",
#             add_line_nos=False,
#             lang="eng"
#         )
        
#         print(f"Raw result keys: {list(result.keys())}")
        
#         # Handle different response structures
#         extraction = None
#         if 'extraction' in result:
#             extraction = result['extraction']
#         elif 'result' in result:
#             extraction = result['result']
#         elif isinstance(result, dict) and 'result_text' in result:
#             extraction = result
#         else:
#             print(f"Unexpected result structure: {type(result)} - {result}")
#             raise ValueError(f"Unexpected response format: {result}")
        
#         # Extract content
#         extracted_text = extraction.get('result_text', '') or extraction.get('text', '') or ''
#         metadata = extraction.get('metadata', {}) or {}
        
#         print(f"Extracted text length: {len(extracted_text)} characters")
#         print(f"Metadata keys: {list(metadata.keys())}")
        
#         # Check if we actually got content
#         if not extracted_text.strip():
#             print("WARNING: No text extracted. This might be a scanned PDF or extraction failure.")
#             # Try OCR mode explicitly
#             print("Retrying with OCR mode...")
#             result_ocr = client.whisper(
#                 file_path=pdf_path,
#                 wait_for_completion=True,
#                 wait_timeout=600,
#                 output_mode="layout_preserving",
#                 pages_to_extract="1-5",  # Test first 5 pages
#                 mode="ocr"  # Force OCR
#             )
#             extraction_ocr = result_ocr.get('extraction', result_ocr)
#             extracted_text = extraction_ocr.get('result_text', '')
#             print(f"OCR attempt - text length: {len(extracted_text)} characters")
        
#         doc_result = {
#             "filename": doc_name,
#             "document_id": doc_id,
#             "file_path": pdf_path,
#             "file_size_mb": round(file_size_mb, 2),
#             "extracted_text": extracted_text[:10000] if len(extracted_text) > 10000 else extracted_text,  # Truncate for storage
#             "full_text_length": len(extracted_text),
#             "pages_processed": metadata.get('pages_processed', 0),
#             "total_pages": metadata.get('total_pages', 0),
#             "extraction_metadata": metadata,
#             "success": bool(extracted_text.strip())
#         }
        
#         all_results['documents'].append(doc_result)
        
#         if doc_result['success']:
#             all_results['processing_summary']['successful'] += 1
#             all_results['processing_summary']['total_pages'] += doc_result['pages_processed']
#             print(f"✓ SUCCESS: {len(extracted_text)} chars, {doc_result['pages_processed']} pages")
#         else:
#             all_results['processing_summary']['failed'] += 1
#             print(f"✗ FAILED: No content extracted")
        
#         # Save individual document
#         doc_output_file = os.path.join(output_dir, f"{doc_id}_extraction.json")
#         with open(doc_output_file, 'w', encoding='utf-8') as f:
#             # Save full text in a separate file if large
#             if len(extracted_text) > 10000:
#                 full_text_file = os.path.join(output_dir, f"{doc_id}_full_text.txt")
#                 with open(full_text_file, 'w', encoding='utf-8') as ft:
#                     ft.write(extracted_text)
#                 doc_result['full_text_file'] = full_text_file
#                 doc_result['extracted_text'] = extracted_text[:1000] + "\n...[truncated]...\n"
            
#             json.dump(doc_result, f, indent=2, ensure_ascii=False)
        
#         # Preview first 500 chars
#         if extracted_text:
#             print(f"Preview:\n{extracted_text[:500]}...\n")
        
#         # Simple table detection in preview
#         if '|' in extracted_text[:2000] or '-' * 10 in extracted_text[:2000]:
#             print("Detected potential table content!")
            
#             # Save full text for table analysis
#             full_text_file = os.path.join(output_dir, f"{doc_id}_full_text_for_tables.txt")
#             with open(full_text_file, 'w', encoding='utf-8') as f:
#                 f.write(extracted_text)
#             print(f"Full text saved for table analysis: {full_text_file}")
        
#         time.sleep(2)  # Rate limiting
        
#     except Exception as e:
#         print(f"ERROR processing {doc_name}: {type(e).__name__}: {str(e)}")
#         all_results['documents'].append({
#             "filename": doc_name, 
#             "error": f"{type(e).__name__}: {str(e)}",
#             "success": False
#         })
#         all_results['processing_summary']['failed'] += 1

# # Save complete results
# complete_file = os.path.join(output_dir, "quarterly_financial_data.json")
# with open(complete_file, 'w', encoding='utf-8') as f:
#     json.dump(all_results, f, indent=2, ensure_ascii=False)

# print(f"\n=== SUMMARY ===")
# print(f"Processed: {all_results['processing_summary']['successful']} successful, {all_results['processing_summary']['failed']} failed")
# print(f"Total pages extracted: {all_results['processing_summary']['total_pages']}")
# print(f"All results saved to: {complete_file}")

# # Show what's in the output directory
# print(f"\nFiles in {output_dir}:")
# for file in os.listdir(output_dir):
#     file_path = os.path.join(output_dir, file)
#     size = os.path.getsize(file_path) / 1024  # KB
#     print(f"  {file} ({size:.1f} KB)")

In [51]:
pip install transformers torch pillow pdf2image

  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
Using cached pdf2image-1.17.0-py3-none-any.whl (11 kB)
Note: you may need to restart the kernel to use updated packages.


In [52]:
from transformers import AutoProcessor, AutoModelForCausalLM
import torch
from pdf2image import convert_from_path
import json

In [54]:
pip install einops timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [timm]1/2 [timm]
Note: you may need to restart the kernel to use updated packages.


In [57]:
import torch
import json
import os
import re
from pathlib import Path
from typing import Dict, List, Any
from PIL import Image
from pdf2image import convert_from_path
import pandas as pd
from transformers import (
    AutoProcessor, 
    AutoModelForCausalLM
)
import warnings
warnings.filterwarnings("ignore")

class Florence2FinancialExtractor:
    def __init__(self, model_name="microsoft/Florence-2-large-ft"):
        """
        Fixed Florence-2 extractor with proper fallback handling
        """
        print("🚀 Loading Microsoft Florence-2...")
        
        # Initialize task tokens FIRST (before any model loading)
        self.task_tokens = {
            "ocr": "<OCR_WITH_REGION>",
            "caption": "<CAPTION>",
            "detailed_caption": "<MORE_DETAILED_CAPTION>",
            "region_caption": "<MORE_DETAILED_CAPTION>",
            "region_detection": "<OD>"
        }
        
        # Try to load the preferred model
        success = False
        try:
            print(f"  🔄 Attempting to load: {model_name}")
            self.processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name, 
                trust_remote_code=True,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                low_cpu_mem_usage=True,
                attn_implementation="eager" if torch.cuda.is_available() else None,
                device_map="auto" if torch.cuda.is_available() else None
            )
            success = True
            print(f"  ✅ Loaded {model_name}")
            
        except Exception as e:
            print(f"  ❌ Failed to load {model_name}: {e}")
            print("  🔄 Trying fallback model...")
        
        # Fallback to base model if large model fails
        if not success:
            try:
                fallback_model = "microsoft/Florence-2-base-ft"
                print(f"  🔄 Loading fallback: {fallback_model}")
                self.processor = AutoProcessor.from_pretrained(fallback_model, trust_remote_code=True)
                self.model = AutoModelForCausalLM.from_pretrained(
                    fallback_model,
                    trust_remote_code=True,
                    torch_dtype=torch.float32,
                    low_cpu_mem_usage=True,
                    attn_implementation="eager"
                )
                success = True
                print(f"  ✅ Loaded fallback: {fallback_model}")
                
            except Exception as e2:
                print(f"  ❌ Fallback failed: {e2}")
                print("  💥 All model loading attempts failed. Check your transformers version.")
                print("  🔧 Try: pip install transformers==4.36.0")
                raise e2
        
        # Final success check
        if success:
            print("✅ Florence-2 loaded successfully!")
            print(f"📱 Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
            print(f"💾 Model dtype: {next(self.model.parameters()).dtype}")
        else:
            raise RuntimeError("Failed to load any Florence-2 model")
    
    def extract_with_florence2(self, image: Image.Image, task: str, text_prompt: str = "") -> str:
        """
        Florence-2 extraction method with proper error handling
        """
        print(f"  🔍 Running task: {task}")
        
        # Use the task token (now guaranteed to exist)
        task_token = self.task_tokens.get(task, "<MORE_DETAILED_CAPTION>")
        
        # Prepare text input
        if text_prompt:
            full_text = f"{task_token}\n{text_prompt}"
        else:
            full_text = task_token
        
        try:
            # Process image and text
            inputs = self.processor(
                text=full_text, 
                images=image, 
                return_tensors="pt"
            )
            
            # Move to device
            device = next(self.model.parameters()).device if hasattr(self.model, 'parameters') else torch.device("cpu")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            # Generate response
            with torch.no_grad():
                generated_ids = self.model.generate(
                    input_ids=inputs["input_ids"],
                    pixel_values=inputs["pixel_values"], 
                    max_new_tokens=512,
                    num_beams=1,  # Single beam for faster processing
                    do_sample=False,
                    pad_token_id=self.processor.tokenizer.eos_token_id,
                    eos_token_id=self.processor.tokenizer.eos_token_id,
                    use_cache=True,
                    output_hidden_states=False,
                    output_attentions=False
                )
            
            # Decode response
            generated_text = self.processor.batch_decode(
                generated_ids.sequences, skip_special_tokens=True
            )[0]
            
            # Clean up response
            if task_token == "<OCR_WITH_REGION>":
                generated_text = re.sub(r'<.*?>', '', generated_text)
            
            return generated_text.strip()
            
        except Exception as e:
            print(f"  ❌ Generation error: {e}")
            return f"Error during extraction: {str(e)}"
    
    def process_page_with_florence(self, pdf_path: str, page_num: int = 2) -> Dict[str, Any]:
        """
        Process specific page with Florence-2
        """
        print(f"\n📄 Processing page {page_num} from {Path(pdf_path).stem}")
        
        try:
            # Convert PDF page to image
            images = convert_from_path(pdf_path, first_page=page_num, last_page=page_num, dpi=150)
            if not images:
                raise ValueError(f"No image generated for page {page_num}")
            
            image = images[0]
            print(f"  🖼️  Image loaded: {image.size}")
            
            # Define tasks for this page
            if page_num == 2:  # Performance highlights page
                tasks = {
                    "performance_highlights": {
                        "task": "detailed_caption",
                        "prompt": "Extract all financial metrics from this TCS quarterly performance highlights page. Look for Revenue (INR/USD), Growth rates (YoY, CC), Margins (Operating, Net), Cash flow, Headcount, Attrition, Order book TCV, Client metrics."
                    },
                    "full_ocr": {
                        "task": "ocr",
                        "prompt": ""
                    }
                }
            elif page_num == 3:  # Growth summary page
                tasks = {
                    "growth_summary": {
                        "task": "detailed_caption",
                        "prompt": "This is a financial growth summary chart. Extract quarterly data: quarters, revenue values, operating income, net income, growth percentages, margins."
                    },
                    "full_ocr": {
                        "task": "ocr",
                        "prompt": ""
                    }
                }
            else:  # Other pages
                tasks = {
                    "document_analysis": {
                        "task": "detailed_caption",
                        "prompt": "Analyze this financial document page and extract any tables, charts, or key metrics."
                    },
                    "full_ocr": {
                        "task": "ocr",
                        "prompt": ""
                    }
                }
            
            results = {}
            
            for task_name, config in tasks.items():
                print(f"  🔄 Running {task_name} task...")
                
                result = self.extract_with_florence2(
                    image=image,
                    task=config["task"],
                    text_prompt=config["prompt"]
                )
                
                results[task_name] = {
                    "raw_response": result,
                    "task": config["task"],
                    "prompt": config["prompt"]
                }
                
                # Parse results based on task type
                if "highlights" in task_name or "growth" in task_name:
                    parsed_metrics = self._parse_performance_metrics(result)
                    results[task_name]["parsed_metrics"] = parsed_metrics
                    if parsed_metrics:
                        print(f"    📊 Extracted {len(parsed_metrics)} metrics")
                
                elif "document" in task_name:
                    parsed_table = self._parse_table_data(result)
                    results[task_name]["parsed_table"] = parsed_table
                    if parsed_table:
                        print(f"    📈 Extracted table with {len(parsed_table)} rows")
                
                # Always do OCR parsing
                if "full_ocr" in task_name:
                    ocr_metrics = self._parse_ocr_metrics(result)
                    results[task_name]["parsed_ocr"] = ocr_metrics
                    if ocr_metrics:
                        print(f"    📝 OCR extracted {len(ocr_metrics)} items")
                
                print(f"    ✅ {task_name} completed ({len(result)} chars)")
                print(f"    📝 Preview: {result[:150]}...")
                print()
            
            return {
                "success": True,
                "page_number": page_num,
                "image_size": image.size,
                "results": results
            }
            
        except Exception as e:
            print(f"  ❌ Error processing page {page_num}: {str(e)}")
            import traceback
            traceback.print_exc()
            return {
                "success": False,
                "page_number": page_num,
                "error": str(e),
                "traceback": traceback.format_exc()
            }
    
    def _parse_performance_metrics(self, text: str) -> Dict[str, Any]:
        """Parse financial metrics from Florence-2 response"""
        metrics = {}
        
        # More robust patterns for financial data
        patterns = {
            "revenue_inr": [
                r"INR Revenue[^{}]*?₹?([,\d]+)\s*(?:Mn|million)", 
                r"₹([,\d]+)\s*Mn", 
                r"Revenue[^{}]*?₹([,\d]+)"
            ],
            "revenue_usd": [
                r"USD Revenue[^{}]*?\$([,\d]+)\s*(?:Mn|million)", 
                r"\$([,\d]+)\s*Mn", 
                r"Revenue[^{}]*?\$([,\d]+)"
            ],
            "yoy_growth": [
                r"(?:up|YoY)[^{}]*?([\d.-]+)%", 
                r"([\d.-]+)%\s*YoY",
                r"Growth[^{}]*?([\d.-]+)%"
            ],
            "cc_growth": [
                r"Constant currency[^{}]*?(?:up|down)[^{}]*?([\d.-]+)%", 
                r"CC[^{}]*?([\d.-]+)%",
                r"Constant[^{}]*?([\d.-]+)%"
            ],
            "operating_margin": [
                r"Operating Margin[^{}]*?([\d.-]+)%", 
                r"Operating[^{}]*?([\d.-]+)%",
                r"Margin[^{}]*?([\d.-]+)%\s*Operating"
            ],
            "net_margin": [
                r"Net Margin[^{}]*?([\d.-]+)%", 
                r"Net[^{}]*?([\d.-]+)%",
                r"Margin[^{}]*?([\d.-]+)%\s*Net"
            ],
            "cash_flow": [
                r"Cash flow[^{}]*?([\d.-]+)%", 
                r"Cash[^{}]*?([\d.-]+)%\s*of",
                r"([\d.-]+)%\s*of net profit"
            ],
            "headcount": [
                r"headcount[^{}]*?:?([,\d]+)", 
                r"([,\d]+)\s*employees?",
                r"Total[^{}]*?([,\d]+)"
            ],
            "attrition": [
                r"attrition[^{}]*?([\d.-]+)%", 
                r"([\d.-]+)%\s*attrition",
                r"([\d.-]+)%\s*LTM"
            ],
            "order_book": [
                r"Order book TCV[^{}]*?\$([,\d.-]+)\s*Bn", 
                r"TCV[^{}]*?\$([,\d.-]+)\s*Bn",
                r"Order[^{}]*?\$([,\d.-]+)\s*Bn"
            ]
        }
        
        for key, pattern_list in patterns.items():
            for pattern in pattern_list:
                matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)
                if matches:
                    # Take the first match and clean it
                    value = re.sub(r'[,\s]', '', matches[0])
                    try:
                        # Handle negative numbers
                        if value.startswith('-'):
                            metrics[key] = float(value)
                        else:
                            metrics[key] = float(value)
                    except ValueError:
                        metrics[key] = value
                    break  # Found a match, stop searching
        
        return metrics
    
    def _parse_ocr_metrics(self, ocr_text: str) -> Dict[str, Any]:
        """Parse metrics directly from OCR output"""
        metrics = {}
        
        # Direct pattern matching from OCR text
        ocr_patterns = {
            "revenue_inr_raw": r"₹\s*([\d,]+)\s*Mn",
            "revenue_usd_raw": r"\$\s*([\d,]+)\s*Mn",
            "growth_pct": r"([\d.-]+)%",
            "margin_pct": r"Margin\s*[\s\w]*\s*([\d.-]+)%",
            "headcount_raw": r"headcount\s*:\s*([\d,]+)",
            "attrition_raw": r"attrition\s*([\d.-]+)%"
        }
        
        for key, pattern in ocr_patterns.items():
            matches = re.findall(pattern, ocr_text, re.IGNORECASE)
            if matches:
                value = re.sub(r'[,\s]', '', matches[0])
                try:
                    metrics[key] = float(value)
                except ValueError:
                    metrics[key] = value
        
        return metrics
    
    def _parse_table_data(self, text: str) -> List[Dict]:
        """Parse table data from Florence-2 response"""
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        
        # Look for table-like structures
        table_rows = []
        current_row = {}
        
        for line in lines:
            # Look for key-value pairs
            if ':' in line and len(line.split(':')) == 2:
                key, value = line.split(':', 1)
                current_row[key.strip()] = value.strip()
            
            # Look for quarter patterns
            quarter_match = re.search(r'(Q[1-4]\s*FY\d{2}-\d{2})', line)
            if quarter_match:
                current_row['quarter'] = quarter_match.group(1)
            
            # Look for numeric values with labels
            if re.search(r'[\$₹]\s*[\d,]+\.?\d*\s*(Mn|Bn)', line):
                # Extract labeled numbers
                num_match = re.search(r'([A-Za-z\s]+?)\s*[\$₹]\s*([\d,]+)\.?\d*\s*(Mn|Bn)', line)
                if num_match:
                    label = num_match.group(1).strip()
                    value = num_match.group(2).replace(',', '')
                    unit = num_match.group(3)
                    current_row[f"{label}_{unit}"] = float(value)
            
            # If we have a complete row, add it
            if len(current_row) >= 2:
                table_rows.append(current_row.copy())
                current_row = {}
        
        return table_rows if table_rows else [{"raw_text": text}]
    
    def process_all_tcs_reports(self, pdf_list: List[str], output_dir: str = "data/output/florence2_results"):
        """
        Process all TCS quarterly reports with Florence-2
        """
        os.makedirs(output_dir, exist_ok=True)
        
        all_results = []
        print(f"\n🔄 Processing {len(pdf_list)} TCS quarterly reports with Florence-2")
        print(f"💻 Using device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
        
        for pdf_path in pdf_list:
            if not os.path.exists(pdf_path):
                print(f"❌ File not found: {pdf_path}")
                continue
            
            doc_name = Path(pdf_path).stem
            print(f"\n{'='*60}")
            print(f"📁 Processing: {doc_name}")
            print(f"{'='*60}")
            
            # Process key pages
            pages_to_process = [2, 3, 4]  # Page 2: Highlights, Page 3: Growth, Page 4: Additional
            doc_results = {
                "document": doc_name,
                "pages_processed": [],
                "summary_metrics": {},
                "extraction_date": pd.Timestamp.now().isoformat()
            }
            
            for page_num in pages_to_process:
                try:
                    page_result = self.process_page_with_florence(pdf_path, page_num)
                    
                    if page_result["success"]:
                        doc_results["pages_processed"].append(page_result)
                        
                        # Aggregate metrics
                        for task_name, task_result in page_result["results"].items():
                            if "parsed_metrics" in task_result and task_result["parsed_metrics"]:
                                doc_results["summary_metrics"].update(task_result["parsed_metrics"])
                    
                    # Save page result
                    page_file = os.path.join(output_dir, f"{doc_name}_page_{page_num}.json")
                    with open(page_file, 'w', encoding='utf-8') as f:
                        json.dump(page_result, f, indent=2, default=str, ensure_ascii=False)
                    
                except Exception as e:
                    print(f"❌ Error processing page {page_num}: {e}")
                    continue
            
            # Save document results
            doc_file = os.path.join(output_dir, f"{doc_name}_complete.json")
            with open(doc_file, 'w', encoding='utf-8') as f:
                json.dump(doc_results, f, indent=2, default=str, ensure_ascii=False)
            
            all_results.append(doc_results)
            
            # Print summary
            metrics = doc_results["summary_metrics"]
            if metrics:
                print(f"  📊 Key metrics extracted ({len(metrics)}):")
                for key, value in list(metrics.items())[:5]:  # Show first 5
                    print(f"    {key}: {value}")
                if len(metrics) > 5:
                    print(f"    ... and {len(metrics)-5} more")
            else:
                print(f"  ⚠ No metrics extracted from {doc_name}")
        
        # Create comparison table
        self._create_comparison_table(all_results, output_dir)
        
        # Save comprehensive summary
        summary_file = os.path.join(output_dir, "tcs_florence2_summary.json")
        with open(summary_file, 'w', encoding='utf-8') as f:
            json.dump(all_results, f, indent=2, default=str, ensure_ascii=False)
        
        print(f"\n🎉 Florence-2 extraction complete!")
        print(f"📁 Results saved to: {output_dir}")
        print(f"📊 Total documents processed: {len([r for r in all_results if 'summary_metrics' in r])}")
        
        return all_results
    
    def _create_comparison_table(self, results: List[Dict], output_dir: str):
        """Create cross-quarter comparison table"""
        comparison_data = []
        
        for result in results:
            metrics = result.get("summary_metrics", {})
            row = {
                "Document": result["document"],
                "Revenue_INR_Mn": metrics.get("revenue_inr", 0),
                "Revenue_USD_Mn": metrics.get("revenue_usd", 0),
                "YoY_Growth_%": metrics.get("yoy_growth", 0),
                "CC_Growth_%": metrics.get("cc_growth", 0),
                "Operating_Margin_%": metrics.get("operating_margin", 0),
                "Net_Margin_%": metrics.get("net_margin", 0),
                "Headcount": metrics.get("headcount", 0),
                "Attrition_%": metrics.get("attrition", 0),
                "Order_Book_Bn": metrics.get("order_book", 0)
            }
            comparison_data.append(row)
        
        if comparison_data:
            df = pd.DataFrame(comparison_data)
            
            # Clean and format numbers
            numeric_cols = ["Revenue_INR_Mn", "Revenue_USD_Mn", "Headcount", "Order_Book_Bn"]
            for col in numeric_cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            percent_cols = ["YoY_Growth_%", "CC_Growth_%", "Operating_Margin_%", "Net_Margin_%", "Attrition_%"]
            for col in percent_cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Save CSV
            csv_file = os.path.join(output_dir, "tcs_quarterly_comparison.csv")
            df.to_csv(csv_file, index=False)
            
            print(f"\n📊 Comparison table created:")
            print(df.round(2).to_string(index=False))
            print(f"📁 CSV saved: {csv_file}")

# Fixed Usage
if __name__ == "__main__":
    # Test the completely fixed extractor
    try:
        print("🧪 Testing Florence-2 compatibility...")
        florence_extractor = Florence2FinancialExtractor()
        
        # Test single page extraction
        sample_pdf = "data/pdfs/Quarter I Ended FY 2025-26.pdf"
        if os.path.exists(sample_pdf):
            print(f"\n🔍 Testing extraction from: {sample_pdf}")
            sample_results = florence_extractor.process_page_with_florence(sample_pdf, page_num=2)
            
            if sample_results["success"]:
                print(f"✅ Page 2 extraction successful!")
                results = sample_results["results"]
                
                # Print performance highlights
                if "performance_highlights" in results:
                    metrics = results["performance_highlights"].get("parsed_metrics", {})
                    print(f"\n📊 Extracted metrics ({len(metrics)}):")
                    for key, value in metrics.items():
                        print(f"  {key}: {value}")
                
                # Print OCR preview
                if "full_ocr" in results:
                    ocr_text = results["full_ocr"]["raw_response"][:300]
                    print(f"\n📝 OCR Preview: {ocr_text}...")
                    
            else:
                print(f"❌ Extraction failed: {sample_results.get('error', 'Unknown error')}")
                if "traceback" in sample_results:
                    print("Traceback:")
                    print(sample_results["traceback"][:500] + "...")
        else:
            print(f"❌ Test PDF not found: {sample_pdf}")
            print("Please ensure your PDFs are in data/pdfs/ directory")
        
        # Process all documents
        pdf_list = [
            "data/pdfs/Quarter I Ended FY 2025-26.pdf",
            "data/pdfs/Quarter III Ended FY 2024-25.pdf",
            "data/pdfs/Quarter IV & Year Ended FY 2024-25.pdf"
        ]
        
        valid_pdfs = [p for p in pdf_list if os.path.exists(p)]
        if valid_pdfs:
            print(f"\n🚀 Processing {len(valid_pdfs)} valid PDFs...")
            results = florence_extractor.process_all_tcs_reports(valid_pdfs)
        else:
            print("❌ No valid PDF files found!")
            print("Available files in data/pdfs/:")
            if os.path.exists("data/pdfs"):
                for file in os.listdir("data/pdfs"):
                    print(f"  - {file}")
            else:
                print("  - Directory 'data/pdfs/' doesn't exist")
                
    except Exception as e:
        print(f"❌ Fatal error: {e}")
        import traceback
        traceback.print_exc()

🧪 Testing Florence-2 compatibility...
🚀 Loading Microsoft Florence-2...
  🔄 Attempting to load: microsoft/Florence-2-large-ft


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

  ❌ Failed to load microsoft/Florence-2-large-ft: 'Florence2ForConditionalGeneration' object has no attribute '_supports_sdpa'
  🔄 Trying fallback model...
  🔄 Loading fallback: microsoft/Florence-2-base-ft


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

...

  ❌ Generation error: Task token <MORE_DETAILED_CAPTION> should be the only token in the text.
    ✅ performance_highlights completed (97 chars)
    📝 Preview: Error during extraction: Task token <MORE_DETAILED_CAPTION> should be the only token in the text....

  🔄 Running full_ocr task...
  🔍 Running task: ocr
  ❌ Generation error: 'NoneType' object has no attribute 'shape'
    ✅ full_ocr completed (67 chars)
    📝 Preview: Error during extraction: 'NoneType' object has no attribute 'shape'...


📄 Processing page 3 from Quarter IV & Year Ended FY 2024-25
  🖼️  Image loaded: (2000, 1125)
  🔄 Running growth_summary task...
  🔍 Running task: detailed_caption
  ❌ Generation error: Task token <MORE_DETAILED_CAPTION> should be the only token in the text.
    ✅ growth_summary completed (97 chars)
    📝 Preview: Error during extraction: Task token <MORE_DETAILED_CAPTION> should be the only token in the text....

  🔄 Running full_ocr task...
  🔍 Running task: ocr


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  ❌ Generation error: 'NoneType' object has no attribute 'shape'
    ✅ full_ocr completed (67 chars)
    📝 Preview: Error during extraction: 'NoneType' object has no attribute 'shape'...


📄 Processing page 4 from Quarter IV & Year Ended FY 2024-25
  🖼️  Image loaded: (2000, 1125)
  🔄 Running document_analysis task...
  🔍 Running task: detailed_caption


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  ❌ Generation error: Task token <MORE_DETAILED_CAPTION> should be the only token in the text.
    📈 Extracted table with 1 rows
    ✅ document_analysis completed (97 chars)
    📝 Preview: Error during extraction: Task token <MORE_DETAILED_CAPTION> should be the only token in the text....

  🔄 Running full_ocr task...
  🔍 Running task: ocr
  ❌ Generation error: 'NoneType' object has no attribute 'shape'
    ✅ full_ocr completed (67 chars)
    📝 Preview: Error during extraction: 'NoneType' object has no attribute 'shape'...

  ⚠ No metrics extracted from Quarter IV & Year Ended FY 2024-25

📊 Comparison table created:
                          Document  Revenue_INR_Mn  Revenue_USD_Mn  YoY_Growth_%  CC_Growth_%  Operating_Margin_%  Net_Margin_%  Headcount  Attrition_%  Order_Book_Bn
        Quarter I Ended FY 2025-26               0               0             0            0                   0             0          0            0              0
      Quarter III Ended FY 2024-25        